In [10]:

import glob
import os
import pandas as pd
import re

#TODO : Change df_date to be abstract
#TODO : Create output directory that has month taken into account i.e. ' output_data/{Month}/ '
#WARNING! DATE IS CURRENTLY HARDCODED for MARCH

path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "Raw_data/*.csv"))

dataframe = 'df-{}'
for f in csv_files:
    df = pd.read_csv(f)

    df['Dates'] = pd.to_datetime(df['Date']).dt.date
    df['Time'] = pd.to_datetime(df['Date']).dt.time
    df1 = df[['Dates','Time','DL(Mbps)','UL(Mbps)']].copy()
    df1['hour'] = pd.to_datetime(df1['Time'].astype(str)).dt.hour
    df2 = df1.loc[df1.hour==9]
    df3 = df1.loc[df1.hour==16]
    df4 = df2[['Dates','DL(Mbps)','UL(Mbps)']] 
    df5 = df3[['Dates','DL(Mbps)','UL(Mbps)']] 

    #Extract DL and UL out of the tables
    df_9am_DL = df4[['Dates', 'DL(Mbps)']]
    df_9am_UL = df4[['Dates', 'UL(Mbps)']]
    df_4pm_DL = df5[['Dates', 'DL(Mbps)']]
    df_4pm_UL = df5[['Dates', 'UL(Mbps)']]

    # We want table to have 1 column split to 2 sub-column                               DL       |      UL
    mux = pd.MultiIndex.from_product([['DL','UL'],['9am','4pm']])                   # =======================
    speedtest_data = pd.DataFrame(None,columns=mux)                                 # 9am | 4pm ||| 9am | 4pm

    #WARNING! DATE IS CURRENTLY HARDCODED for MARCH
    df_date = pd.DataFrame({'Date':pd.date_range(start='2022-03-01',end='2022-03-31')})
    speedtest_data1 = pd.concat([df_date,speedtest_data],axis=1)

    speedtest_data1[('DL','9am')] = pd.Series(df_9am_DL['DL(Mbps)'].values)
    speedtest_data1[('DL','4pm')] = pd.Series(df_4pm_DL['DL(Mbps)'].values)
    speedtest_data1[('UL','9am')] = pd.Series(df_9am_UL['UL(Mbps)'].values)
    speedtest_data1[('UL','4pm')] = pd.Series(df_4pm_UL['UL(Mbps)'].values)

    #remove 'MBps unit from data'
    speedtest_data1[('DL','9am')] = speedtest_data1[('DL','9am')].str.replace(r'\D+','',regex=True)
    speedtest_data1[('DL','4pm')] = speedtest_data1[('DL','4pm')].str.replace(r'\D+','',regex=True)
    speedtest_data1[('UL','9am')] = speedtest_data1[('UL','9am')].str.replace(r'\D+','',regex=True)
    speedtest_data1[('UL','4pm')] = speedtest_data1[('UL','4pm')].str.replace(r'\D+','',regex=True)

    #output the files as csv
    file_name = f.split("/")[-1]
    output_file = '{}_output'

    # check if a directory exist, if not create it
    if not os.path.isdir('output_data'):
        os.makedirs('output_data')

    speedtest_data1.to_csv(path+'/output_data/'+output_file.format(file_name), sep='\t', encoding='utf=8')
